In [2]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
import pandas as pd

In [3]:
# search for a player
lebron = players.find_players_by_full_name("LeBron James")[0]
lebron_id = lebron["id"]

print(lebron_id)


2544


In [4]:
shots = shotchartdetail.ShotChartDetail(
    team_id=0,                    # 0 = all teams
    player_id=lebron_id,
    season_type_all_star="Regular Season",
    season_nullable="2023-24",
    context_measure_simple="FGA"  # field goal attempts
)


In [5]:
df = shots.get_data_frames()[0]
df.head()


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022300015,7,2544,LeBron James,1610612747,Los Angeles Lakers,1,11,40,...,Center(C),16-24 ft.,17,-47,172,1,0,20231110,PHX,LAL
1,Shot Chart Detail,0022300015,11,2544,LeBron James,1610612747,Los Angeles Lakers,1,11,12,...,Center(C),Less Than 8 ft.,0,2,5,1,1,20231110,PHX,LAL
2,Shot Chart Detail,0022300015,20,2544,LeBron James,1610612747,Los Angeles Lakers,1,10,1,...,Left Side(L),8-16 ft.,12,-119,32,1,0,20231110,PHX,LAL
3,Shot Chart Detail,0022300015,36,2544,LeBron James,1610612747,Los Angeles Lakers,1,9,0,...,Center(C),Less Than 8 ft.,0,1,8,1,1,20231110,PHX,LAL
4,Shot Chart Detail,0022300015,72,2544,LeBron James,1610612747,Los Angeles Lakers,1,5,18,...,Left Side(L),8-16 ft.,12,-82,89,1,0,20231110,PHX,LAL


In [6]:
# % of shots that are 3s
three_rate = (df["SHOT_TYPE"] == "3PT Field Goal").mean()

# FG% by distance
fg_by_dist = df.groupby("SHOT_DISTANCE")["SHOT_MADE_FLAG"].mean()

print("3PT Rate:", three_rate)


3PT Rate: 0.2860520094562648


In [7]:
import time
import pandas as pd

from nba_api.stats.static import players
from nba_api.stats.endpoints import leaguegamefinder, shotchartdetail

# --- 1) season helper: last 10 completed Finals seasons (2015-16 .. 2024-25) ---
def last_10_completed_finals_seasons():
    # 2024-25 is the last completed season before Jan 2026
    end_start_year = 2025
    start_start_year = end_start_year - 9  # 10 seasons total
    return [f"{y}-{str(y+1)[-2:]}" for y in range(start_start_year, end_start_year + 1)]

# --- 2) robust request wrapper (rate limit + retry) ---
def safe_call(fn, tries=4, base_sleep=1.0):
    for i in range(tries):
        try:
            return fn()
        except Exception as e:
            if i == tries - 1:
                raise
            time.sleep(base_sleep * (2 ** i))

# --- 3) find player_id from name ---
def get_player_id(full_name: str) -> int:
    matches = players.find_players_by_full_name(full_name)
    if not matches:
        raise ValueError(f"No player found for name: {full_name}")
    # best match is usually first
    return matches[0]["id"]

# --- 4) Finals GAME_IDs the player appeared in (per season) ---
def get_player_finals_game_ids(player_id: int, season: str, sleep_s=0.8):
    # LeagueGameFinder supports player_id_nullable, season_type_nullable, and po_round_nullable. :contentReference[oaicite:1]{index=1}
    def _call():
        return leaguegamefinder.LeagueGameFinder(
            player_id_nullable=player_id,
            season_nullable=season,
            season_type_nullable="Playoffs",
            po_round_nullable=4,  # 4 = Finals round
        ).get_data_frames()[0]

    games = safe_call(_call)
    time.sleep(sleep_s)

    # If player made Finals that year, this will have 4–7 rows (one per game)
    if games.empty:
        return []

    # Extra safety: only keep actual playoff game ids (typically start with '004' for playoffs)
    # (Not strictly necessary since we asked Playoffs + PO round.)
    game_ids = sorted(games["GAME_ID"].unique().tolist())
    return game_ids

# --- 5) All Finals shot attempts for the player (per game) ---
def get_player_shots_for_game(player_id: int, season: str, game_id: str, sleep_s=0.8):
    # ShotChartDetail supports player_id, season_type_all_star, season_nullable, game_id_nullable, context_measure_simple. :contentReference[oaicite:2]{index=2}
    def _call():
        return shotchartdetail.ShotChartDetail(
            team_id=0,
            player_id=player_id,
            season_nullable=season,
            season_type_all_star="Playoffs",
            game_id_nullable=game_id,
            context_measure_simple="FGA"
        ).get_data_frames()[0]

    df = safe_call(_call)
    time.sleep(sleep_s)

    if not df.empty:
        df["SEASON"] = season
        df["FINALS_GAME_ID"] = game_id
    return df

# --- 6) full pipeline: ANY player -> last 10 seasons -> Finals only -> all FGAs ---
def finals_shots_last_10_seasons(player_full_name: str):
    player_id = get_player_id(player_full_name)
    seasons = last_10_completed_finals_seasons()

    out = []
    finals_games_index = []  # optional: store which seasons they reached Finals

    for season in seasons:
        game_ids = get_player_finals_game_ids(player_id, season)
        if game_ids:
            finals_games_index.append((season, len(game_ids)))

        for gid in game_ids:
            df_game = get_player_shots_for_game(player_id, season, gid)
            if not df_game.empty:
                out.append(df_game)

    shots_df = pd.concat(out, ignore_index=True) if out else pd.DataFrame()
    finals_summary = pd.DataFrame(finals_games_index, columns=["SEASON", "FINALS_GAMES_FOUND"])
    return shots_df, finals_summary


In [ ]:
# ------------------ USAGE ------------------
shots_df, finals_summary = finals_shots_last_10_seasons("LeBron James")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)

    SEASON  FINALS_GAMES_FOUND
0  2016-17                  18
1  2017-18                  22
2  2019-20                  21
3  2020-21                   6
4  2022-23                  16
5  2023-24                   5
6  2024-25                   5
Total Finals shot attempts rows: 1872


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041600111,97,2544,LeBron James,1610612739,Cleveland Cavaliers,1,2,33,...,0,-7,-1,1,0,20170415,CLE,IND,2016-17,0041600111
1,Shot Chart Detail,0041600111,113,2544,LeBron James,1610612739,Cleveland Cavaliers,1,1,40,...,0,-7,-6,1,1,20170415,CLE,IND,2016-17,0041600111
2,Shot Chart Detail,0041600111,141,2544,LeBron James,1610612739,Cleveland Cavaliers,2,10,48,...,0,-2,-1,1,1,20170415,CLE,IND,2016-17,0041600111
3,Shot Chart Detail,0041600111,159,2544,LeBron James,1610612739,Cleveland Cavaliers,2,9,14,...,0,0,3,1,1,20170415,CLE,IND,2016-17,0041600111
4,Shot Chart Detail,0041600111,190,2544,LeBron James,1610612739,Cleveland Cavaliers,2,6,30,...,0,0,1,1,1,20170415,CLE,IND,2016-17,0041600111
5,Shot Chart Detail,0041600111,280,2544,LeBron James,1610612739,Cleveland Cavaliers,2,0,1,...,0,-7,-1,1,1,20170415,CLE,IND,2016-17,0041600111
6,Shot Chart Detail,0041600111,290,2544,LeBron James,1610612739,Cleveland Cavaliers,3,11,16,...,7,-50,61,1,1,20170415,CLE,IND,2016-17,0041600111
7,Shot Chart Detail,0041600111,296,2544,LeBron James,1610612739,Cleveland Cavaliers,3,10,50,...,23,-235,-6,1,1,20170415,CLE,IND,2016-17,0041600111
8,Shot Chart Detail,0041600111,318,2544,LeBron James,1610612739,Cleveland Cavaliers,3,7,3,...,25,-173,190,1,1,20170415,CLE,IND,2016-17,0041600111
9,Shot Chart Detail,0041600111,346,2544,LeBron James,1610612739,Cleveland Cavaliers,3,5,5,...,11,22,115,1,0,20170415,CLE,IND,2016-17,0041600111


In [3]:
shots_df, finals_summary = finals_shots_last_10_seasons("Devin Booker")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2020-21                  22
1  2021-22                  10
2  2022-23                  11
3  2023-24                   4
Total Finals shot attempts rows: 955


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0042000151,9,1626164,Devin Booker,1610612756,Phoenix Suns,1,11,28,...,28,-202,196,1,0,20210523,PHX,LAL,2020-21,0042000151
1,Shot Chart Detail,0042000151,13,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,59,...,1,-11,1,1,0,20210523,PHX,LAL,2020-21,0042000151
2,Shot Chart Detail,0042000151,17,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,16,...,23,230,-13,1,1,20210523,PHX,LAL,2020-21,0042000151
3,Shot Chart Detail,0042000151,71,1626164,Devin Booker,1610612756,Phoenix Suns,1,5,23,...,0,-2,-3,1,1,20210523,PHX,LAL,2020-21,0042000151
4,Shot Chart Detail,0042000151,90,1626164,Devin Booker,1610612756,Phoenix Suns,1,3,46,...,0,8,-3,1,1,20210523,PHX,LAL,2020-21,0042000151
5,Shot Chart Detail,0042000151,127,1626164,Devin Booker,1610612756,Phoenix Suns,1,0,42,...,7,74,16,1,1,20210523,PHX,LAL,2020-21,0042000151
6,Shot Chart Detail,0042000151,130,1626164,Devin Booker,1610612756,Phoenix Suns,1,0,33,...,30,55,302,1,0,20210523,PHX,LAL,2020-21,0042000151
7,Shot Chart Detail,0042000151,209,1626164,Devin Booker,1610612756,Phoenix Suns,2,7,24,...,20,79,190,1,1,20210523,PHX,LAL,2020-21,0042000151
8,Shot Chart Detail,0042000151,220,1626164,Devin Booker,1610612756,Phoenix Suns,2,6,52,...,16,-160,34,1,0,20210523,PHX,LAL,2020-21,0042000151
9,Shot Chart Detail,0042000151,226,1626164,Devin Booker,1610612756,Phoenix Suns,2,6,11,...,6,-14,63,1,0,20210523,PHX,LAL,2020-21,0042000151


In [4]:
shots_df, finals_summary = finals_shots_last_10_seasons("Grant Williams")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2019-20                  17
1  2020-21                   5
2  2021-22                  24
3  2022-23                  15
Total Finals shot attempts rows: 241


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041900122,84,1629684,Grant Williams,1610612738,Boston Celtics,1,4,36,...,26,-86,252,1,1,20200819,BOS,PHI,2019-20,0041900122
1,Shot Chart Detail,0041900122,388,1629684,Grant Williams,1610612738,Boston Celtics,3,4,37,...,26,-153,221,1,1,20200819,BOS,PHI,2019-20,0041900122
2,Shot Chart Detail,0041900123,128,1629684,Grant Williams,1610612738,Boston Celtics,1,2,56,...,5,43,26,1,0,20200821,PHI,BOS,2019-20,0041900123
3,Shot Chart Detail,0041900123,471,1629684,Grant Williams,1610612738,Boston Celtics,3,4,23,...,2,8,25,1,1,20200821,PHI,BOS,2019-20,0041900123
4,Shot Chart Detail,0041900123,483,1629684,Grant Williams,1610612738,Boston Celtics,3,3,14,...,24,1,248,1,1,20200821,PHI,BOS,2019-20,0041900123
5,Shot Chart Detail,0041900124,452,1629684,Grant Williams,1610612738,Boston Celtics,3,3,5,...,23,-234,0,1,1,20200823,PHI,BOS,2019-20,0041900124
6,Shot Chart Detail,0041900212,278,1629684,Grant Williams,1610612738,Boston Celtics,2,0,45,...,1,8,7,1,1,20200901,TOR,BOS,2019-20,0041900212
7,Shot Chart Detail,0041900212,399,1629684,Grant Williams,1610612738,Boston Celtics,3,4,49,...,2,22,19,1,0,20200901,TOR,BOS,2019-20,0041900212
8,Shot Chart Detail,0041900213,159,1629684,Grant Williams,1610612738,Boston Celtics,1,1,52,...,22,226,-5,1,0,20200903,BOS,TOR,2019-20,0041900213
9,Shot Chart Detail,0041900213,440,1629684,Grant Williams,1610612738,Boston Celtics,3,2,31,...,11,100,63,1,0,20200903,BOS,TOR,2019-20,0041900213


In [5]:
shots_df, finals_summary = finals_shots_last_10_seasons("Giannis Antetokounmpo")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)

    SEASON  FINALS_GAMES_FOUND
0  2016-17                   6
1  2017-18                   7
2  2018-19                  15
3  2019-20                   9
4  2020-21                  21
5  2021-22                  12
6  2022-23                   3
7  2024-25                   5
Total Finals shot attempts rows: 1531


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041600121,10,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,10,34,...,0,0,1,1,1,20170415,TOR,MIL,2016-17,0041600121
1,Shot Chart Detail,0041600121,26,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,9,5,...,2,-11,21,1,0,20170415,TOR,MIL,2016-17,0041600121
2,Shot Chart Detail,0041600121,41,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,7,59,...,26,-165,203,1,0,20170415,TOR,MIL,2016-17,0041600121
3,Shot Chart Detail,0041600121,51,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,7,20,...,2,12,16,1,1,20170415,TOR,MIL,2016-17,0041600121
4,Shot Chart Detail,0041600121,62,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,5,51,...,0,0,1,1,1,20170415,TOR,MIL,2016-17,0041600121
5,Shot Chart Detail,0041600121,126,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,2,11,18,...,1,15,-6,1,1,20170415,TOR,MIL,2016-17,0041600121
6,Shot Chart Detail,0041600121,138,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,2,10,13,...,18,-88,160,1,1,20170415,TOR,MIL,2016-17,0041600121
7,Shot Chart Detail,0041600121,146,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,2,9,4,...,1,-12,8,1,0,20170415,TOR,MIL,2016-17,0041600121
8,Shot Chart Detail,0041600121,156,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,2,8,3,...,0,0,1,1,1,20170415,TOR,MIL,2016-17,0041600121
9,Shot Chart Detail,0041600121,182,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,2,6,27,...,0,0,1,1,1,20170415,TOR,MIL,2016-17,0041600121


In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Nikola Jokic")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
shots_df, finals_summary = finals_shots_last_10_seasons("Klay Thompson")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2016-17                  17
1  2017-18                  21
2  2018-19                  21
3  2021-22                  22
4  2022-23                  13
Total Finals shot attempts rows: 1526


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041600141,29,202691,Klay Thompson,1610612744,Golden State Warriors,1,9,19,...,17.0,-1.0,174.0,1,0,20170416,GSW,POR,2016-17,0041600141
1,Shot Chart Detail,0041600141,33,202691,Klay Thompson,1610612744,Golden State Warriors,1,8,58,...,26.0,153.0,218.0,1,0,20170416,GSW,POR,2016-17,0041600141
2,Shot Chart Detail,0041600141,56,202691,Klay Thompson,1610612744,Golden State Warriors,1,7,40,...,27.0,-181.0,203.0,1,0,20170416,GSW,POR,2016-17,0041600141
3,Shot Chart Detail,0041600141,178,202691,Klay Thompson,1610612744,Golden State Warriors,2,9,7,...,19.0,142.0,127.0,1,0,20170416,GSW,POR,2016-17,0041600141
4,Shot Chart Detail,0041600141,181,202691,Klay Thompson,1610612744,Golden State Warriors,2,8,49,...,1.0,9.0,11.0,1,1,20170416,GSW,POR,2016-17,0041600141
5,Shot Chart Detail,0041600141,184,202691,Klay Thompson,1610612744,Golden State Warriors,2,8,17,...,25.0,53.0,252.0,1,1,20170416,GSW,POR,2016-17,0041600141
6,Shot Chart Detail,0041600141,235,202691,Klay Thompson,1610612744,Golden State Warriors,2,3,15,...,13.0,91.0,101.0,1,1,20170416,GSW,POR,2016-17,0041600141
7,Shot Chart Detail,0041600141,253,202691,Klay Thompson,1610612744,Golden State Warriors,2,1,32,...,26.0,159.0,213.0,1,0,20170416,GSW,POR,2016-17,0041600141
8,Shot Chart Detail,0041600141,256,202691,Klay Thompson,1610612744,Golden State Warriors,2,0,57,...,19.0,96.0,174.0,1,0,20170416,GSW,POR,2016-17,0041600141
9,Shot Chart Detail,0041600141,309,202691,Klay Thompson,1610612744,Golden State Warriors,3,6,36,...,8.0,-11.0,87.0,1,1,20170416,GSW,POR,2016-17,0041600141


In [9]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jayson Tatum")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2017-18                  19
1  2018-19                   9
2  2019-20                  17
3  2020-21                   5
4  2021-22                  24
5  2022-23                  20
6  2023-24                  19
7  2024-25                   8
Total Finals shot attempts rows: 2241


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041700111,22,1628369,Jayson Tatum,1610612738,Boston Celtics,1,9,59,...,18,-30,183,1,1,20180415,BOS,MIL,2017-18,0041700111
1,Shot Chart Detail,0041700111,39,1628369,Jayson Tatum,1610612738,Boston Celtics,1,8,26,...,0,0,7,1,1,20180415,BOS,MIL,2017-18,0041700111
2,Shot Chart Detail,0041700111,60,1628369,Jayson Tatum,1610612738,Boston Celtics,1,5,10,...,13,-14,132,1,1,20180415,BOS,MIL,2017-18,0041700111
3,Shot Chart Detail,0041700111,68,1628369,Jayson Tatum,1610612738,Boston Celtics,1,3,56,...,2,16,12,1,1,20180415,BOS,MIL,2017-18,0041700111
4,Shot Chart Detail,0041700111,181,1628369,Jayson Tatum,1610612738,Boston Celtics,2,9,17,...,15,30,153,1,0,20180415,BOS,MIL,2017-18,0041700111
5,Shot Chart Detail,0041700111,204,1628369,Jayson Tatum,1610612738,Boston Celtics,2,7,24,...,20,-71,195,1,0,20180415,BOS,MIL,2017-18,0041700111
6,Shot Chart Detail,0041700111,259,1628369,Jayson Tatum,1610612738,Boston Celtics,2,3,33,...,4,-34,29,1,0,20180415,BOS,MIL,2017-18,0041700111
7,Shot Chart Detail,0041700111,297,1628369,Jayson Tatum,1610612738,Boston Celtics,2,0,56,...,22,226,1,1,0,20180415,BOS,MIL,2017-18,0041700111
8,Shot Chart Detail,0041700111,323,1628369,Jayson Tatum,1610612738,Boston Celtics,3,10,49,...,1,-14,10,1,0,20180415,BOS,MIL,2017-18,0041700111
9,Shot Chart Detail,0041700111,347,1628369,Jayson Tatum,1610612738,Boston Celtics,3,8,30,...,26,119,237,1,0,20180415,BOS,MIL,2017-18,0041700111


In [10]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jamal Murray")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2018-19                  14
1  2019-20                  19
2  2022-23                  20
3  2023-24                  12
4  2024-25                  14
Total Finals shot attempts rows: 1529


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041800151,9,1627750,Jamal Murray,1610612743,Denver Nuggets,1,11,40,...,25,-159,195,1,0,20190413,DEN,SAS,2018-19,0041800151
1,Shot Chart Detail,0041800151,44,1627750,Jamal Murray,1610612743,Denver Nuggets,1,9,13,...,15,-96,120,1,1,20190413,DEN,SAS,2018-19,0041800151
2,Shot Chart Detail,0041800151,58,1627750,Jamal Murray,1610612743,Denver Nuggets,1,7,48,...,23,204,126,1,0,20190413,DEN,SAS,2018-19,0041800151
3,Shot Chart Detail,0041800151,61,1627750,Jamal Murray,1610612743,Denver Nuggets,1,7,44,...,23,193,126,1,0,20190413,DEN,SAS,2018-19,0041800151
4,Shot Chart Detail,0041800151,177,1627750,Jamal Murray,1610612743,Denver Nuggets,1,0,2,...,17,-33,168,1,0,20190413,DEN,SAS,2018-19,0041800151
5,Shot Chart Detail,0041800151,271,1627750,Jamal Murray,1610612743,Denver Nuggets,2,5,7,...,14,-87,115,1,1,20190413,DEN,SAS,2018-19,0041800151
6,Shot Chart Detail,0041800151,302,1627750,Jamal Murray,1610612743,Denver Nuggets,2,2,24,...,14,-77,121,1,1,20190413,DEN,SAS,2018-19,0041800151
7,Shot Chart Detail,0041800151,305,1627750,Jamal Murray,1610612743,Denver Nuggets,2,2,5,...,17,-116,127,1,0,20190413,DEN,SAS,2018-19,0041800151
8,Shot Chart Detail,0041800151,312,1627750,Jamal Murray,1610612743,Denver Nuggets,2,1,6,...,9,-43,85,1,0,20190413,DEN,SAS,2018-19,0041800151
9,Shot Chart Detail,0041800151,317,1627750,Jamal Murray,1610612743,Denver Nuggets,2,0,32,...,12,-56,109,1,1,20190413,DEN,SAS,2018-19,0041800151


In [11]:
shots_df, finals_summary = finals_shots_last_10_seasons("Bruce Brown")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2018-19                   4
1  2020-21                  12
2  2021-22                   4
3  2022-23                  20
Total Finals shot attempts rows: 306


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041800101,408,1628971,Bruce Brown,1610612765,Detroit Pistons,3,8,16,...,24,244,48,1,0,20190414,MIL,DET,2018-19,0041800101
1,Shot Chart Detail,0041800101,417,1628971,Bruce Brown,1610612765,Detroit Pistons,3,7,9,...,4,34,29,1,1,20190414,MIL,DET,2018-19,0041800101
2,Shot Chart Detail,0041800102,204,1628971,Bruce Brown,1610612765,Detroit Pistons,2,11,4,...,1,14,10,1,0,20190417,MIL,DET,2018-19,0041800102
3,Shot Chart Detail,0041800102,248,1628971,Bruce Brown,1610612765,Detroit Pistons,2,8,24,...,6,-57,39,1,0,20190417,MIL,DET,2018-19,0041800102
4,Shot Chart Detail,0041800102,256,1628971,Bruce Brown,1610612765,Detroit Pistons,2,7,49,...,1,-8,15,1,1,20190417,MIL,DET,2018-19,0041800102
5,Shot Chart Detail,0041800102,526,1628971,Bruce Brown,1610612765,Detroit Pistons,4,11,16,...,25,241,94,1,1,20190417,MIL,DET,2018-19,0041800102
6,Shot Chart Detail,0041800103,118,1628971,Bruce Brown,1610612765,Detroit Pistons,1,2,34,...,25,-130,214,1,0,20190420,DET,MIL,2018-19,0041800103
7,Shot Chart Detail,0041800103,134,1628971,Bruce Brown,1610612765,Detroit Pistons,1,1,42,...,5,34,48,1,0,20190420,DET,MIL,2018-19,0041800103
8,Shot Chart Detail,0041800103,534,1628971,Bruce Brown,1610612765,Detroit Pistons,3,2,21,...,1,10,4,1,1,20190420,DET,MIL,2018-19,0041800103
9,Shot Chart Detail,0041800104,23,1628971,Bruce Brown,1610612765,Detroit Pistons,1,10,1,...,22,-227,13,1,0,20190422,DET,MIL,2018-19,0041800104


In [13]:
shots_df, finals_summary = finals_shots_last_10_seasons("Kevon Looney")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
shots_df, finals_summary = finals_shots_last_10_seasons("Aaron Gordon")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [15]:
shots_df, finals_summary = finals_shots_last_10_seasons("Derrick White")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("PJ Tucker")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


ValueError: No player found for name: Pj Tucker

In [10]:
shots_df, finals_summary = finals_shots_last_10_seasons("Danny Green")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2016-17                  16
1  2017-18                   5
2  2018-19                  24
3  2019-20                  21
4  2020-21                   8
5  2021-22                  12
6  2022-23                   4
Total Finals shot attempts rows: 608


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041600151,7,201980,Danny Green,1610612759,San Antonio Spurs,1,10,21,...,23,233,11,1,0,20170415,SAS,MEM,2016-17,0041600151
1,Shot Chart Detail,0041600151,266,201980,Danny Green,1610612759,San Antonio Spurs,3,7,25,...,27,148,234,1,0,20170415,SAS,MEM,2016-17,0041600151
2,Shot Chart Detail,0041600152,71,201980,Danny Green,1610612759,San Antonio Spurs,1,5,26,...,24,-150,188,1,1,20170417,SAS,MEM,2016-17,0041600152
3,Shot Chart Detail,0041600152,166,201980,Danny Green,1610612759,San Antonio Spurs,2,8,43,...,27,-25,272,1,1,20170417,SAS,MEM,2016-17,0041600152
4,Shot Chart Detail,0041600152,301,201980,Danny Green,1610612759,San Antonio Spurs,3,8,51,...,24,151,193,1,1,20170417,SAS,MEM,2016-17,0041600152
5,Shot Chart Detail,0041600152,392,201980,Danny Green,1610612759,San Antonio Spurs,4,10,53,...,25,-178,180,1,0,20170417,SAS,MEM,2016-17,0041600152
6,Shot Chart Detail,0041600152,436,201980,Danny Green,1610612759,San Antonio Spurs,4,6,28,...,26,197,183,1,1,20170417,SAS,MEM,2016-17,0041600152
7,Shot Chart Detail,0041600153,51,201980,Danny Green,1610612759,San Antonio Spurs,1,6,45,...,1,15,-5,1,1,20170420,MEM,SAS,2016-17,0041600153
8,Shot Chart Detail,0041600153,173,201980,Danny Green,1610612759,San Antonio Spurs,2,4,53,...,25,-217,129,1,0,20170420,MEM,SAS,2016-17,0041600153
9,Shot Chart Detail,0041600153,189,201980,Danny Green,1610612759,San Antonio Spurs,2,3,0,...,1,-14,-5,1,1,20170420,MEM,SAS,2016-17,0041600153


In [17]:
shots_df, finals_summary = finals_shots_last_10_seasons("Kentavious Caldwell-Pope")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2019-20                  21
1  2020-21                   5
2  2022-23                  20
3  2023-24                  12
4  2024-25                   5
Total Finals shot attempts rows: 496


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041900141,55,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,7,59,...,2,-6,20,1,0,20200818,LAL,POR,2019-20,0041900141
1,Shot Chart Detail,0041900141,98,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,5,48,...,25,123,220,1,0,20200818,LAL,POR,2019-20,0041900141
2,Shot Chart Detail,0041900141,132,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,3,48,...,22,225,15,1,0,20200818,LAL,POR,2019-20,0041900141
3,Shot Chart Detail,0041900141,417,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,3,10,34,...,23,-231,-20,1,0,20200818,LAL,POR,2019-20,0041900141
4,Shot Chart Detail,0041900141,450,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,3,8,39,...,23,230,13,1,0,20200818,LAL,POR,2019-20,0041900141
5,Shot Chart Detail,0041900141,456,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,3,8,8,...,15,152,11,1,0,20200818,LAL,POR,2019-20,0041900141
6,Shot Chart Detail,0041900141,520,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,3,3,1,...,25,-169,190,1,0,20200818,LAL,POR,2019-20,0041900141
7,Shot Chart Detail,0041900141,682,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,4,3,58,...,2,28,6,1,0,20200818,LAL,POR,2019-20,0041900141
8,Shot Chart Detail,0041900141,699,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,4,2,53,...,6,-40,45,1,0,20200818,LAL,POR,2019-20,0041900141
9,Shot Chart Detail,0041900142,38,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,8,4,...,26,-14,262,1,1,20200820,LAL,POR,2019-20,0041900142


In [12]:
shots_df, finals_summary = finals_shots_last_10_seasons("Draymond Green")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2016-17                  17
1  2017-18                  21
2  2018-19                  22
3  2021-22                  22
4  2022-23                  12
5  2024-25                  12
Total Finals shot attempts rows: 925


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041600141,2,203110,Draymond Green,1610612744,Golden State Warriors,1,11,46,...,9,91,26,1,0,20170416,GSW,POR,2016-17,0041600141
1,Shot Chart Detail,0041600141,143,203110,Draymond Green,1610612744,Golden State Warriors,1,0,37,...,28,127,257,1,1,20170416,GSW,POR,2016-17,0041600141
2,Shot Chart Detail,0041600141,164,203110,Draymond Green,1610612744,Golden State Warriors,2,10,56,...,25,-68,247,1,1,20170416,GSW,POR,2016-17,0041600141
3,Shot Chart Detail,0041600141,288,203110,Draymond Green,1610612744,Golden State Warriors,3,10,9,...,2,19,21,1,1,20170416,GSW,POR,2016-17,0041600141
4,Shot Chart Detail,0041600141,401,203110,Draymond Green,1610612744,Golden State Warriors,4,11,42,...,26,-102,247,1,1,20170416,GSW,POR,2016-17,0041600141
5,Shot Chart Detail,0041600141,458,203110,Draymond Green,1610612744,Golden State Warriors,4,6,53,...,0,0,1,1,0,20170416,GSW,POR,2016-17,0041600141
6,Shot Chart Detail,0041600141,485,203110,Draymond Green,1610612744,Golden State Warriors,4,4,58,...,0,2,7,1,1,20170416,GSW,POR,2016-17,0041600141
7,Shot Chart Detail,0041600141,504,203110,Draymond Green,1610612744,Golden State Warriors,4,2,58,...,23,-230,8,1,0,20170416,GSW,POR,2016-17,0041600141
8,Shot Chart Detail,0041600141,507,203110,Draymond Green,1610612744,Golden State Warriors,4,2,44,...,1,9,16,1,1,20170416,GSW,POR,2016-17,0041600141
9,Shot Chart Detail,0041600141,547,203110,Draymond Green,1610612744,Golden State Warriors,4,0,22,...,2,-1,21,1,0,20170416,GSW,POR,2016-17,0041600141


In [13]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jrue Holiday")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2017-18                   9
1  2020-21                  23
2  2021-22                  12
3  2022-23                   5
4  2023-24                  19
5  2024-25                   8
Total Finals shot attempts rows: 1132


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041700161,14,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,10,48,...,16,-150,56,1,0,20180414,POR,NOP,2017-18,0041700161
1,Shot Chart Detail,0041700161,18,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,10,16,...,25,89,240,1,0,20180414,POR,NOP,2017-18,0041700161
2,Shot Chart Detail,0041700161,39,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,8,48,...,6,38,55,1,0,20180414,POR,NOP,2017-18,0041700161
3,Shot Chart Detail,0041700161,59,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,7,25,...,2,20,14,1,1,20180414,POR,NOP,2017-18,0041700161
4,Shot Chart Detail,0041700161,69,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,6,25,...,25,225,117,1,1,20180414,POR,NOP,2017-18,0041700161
5,Shot Chart Detail,0041700161,72,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,5,57,...,24,-90,230,1,0,20180414,POR,NOP,2017-18,0041700161
6,Shot Chart Detail,0041700161,107,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,3,13,...,14,-105,104,1,1,20180414,POR,NOP,2017-18,0041700161
7,Shot Chart Detail,0041700161,123,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,2,7,...,14,-101,101,1,0,20180414,POR,NOP,2017-18,0041700161
8,Shot Chart Detail,0041700161,136,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,1,16,...,1,8,6,1,1,20180414,POR,NOP,2017-18,0041700161
9,Shot Chart Detail,0041700161,145,201950,Jrue Holiday,1610612740,New Orleans Pelicans,1,0,0,...,1,-13,1,1,1,20180414,POR,NOP,2017-18,0041700161


In [14]:
shots_df, finals_summary = finals_shots_last_10_seasons("Andrew wiggins")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2017-18                   5
1  2021-22                  22
2  2022-23                  13
3  2024-25                   4
Total Finals shot attempts rows: 597


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041700141,18,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,1,10,29,...,2,-20,13,1,1,20180415,HOU,MIN,2017-18,0041700141
1,Shot Chart Detail,0041700141,65,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,1,6,17,...,1,-11,13,1,1,20180415,HOU,MIN,2017-18,0041700141
2,Shot Chart Detail,0041700141,76,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,1,5,47,...,2,-20,6,1,1,20180415,HOU,MIN,2017-18,0041700141
3,Shot Chart Detail,0041700141,80,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,1,5,10,...,26,109,240,1,0,20180415,HOU,MIN,2017-18,0041700141
4,Shot Chart Detail,0041700141,90,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,1,4,36,...,12,-43,117,1,1,20180415,HOU,MIN,2017-18,0041700141
5,Shot Chart Detail,0041700141,101,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,1,3,34,...,0,-2,-4,1,0,20180415,HOU,MIN,2017-18,0041700141
6,Shot Chart Detail,0041700141,132,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,1,1,44,...,9,-91,20,1,0,20180415,HOU,MIN,2017-18,0041700141
7,Shot Chart Detail,0041700141,262,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,2,5,4,...,3,-11,37,1,0,20180415,HOU,MIN,2017-18,0041700141
8,Shot Chart Detail,0041700141,294,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,2,1,32,...,26,-222,140,1,0,20180415,HOU,MIN,2017-18,0041700141
9,Shot Chart Detail,0041700141,296,203952,Andrew Wiggins,1610612750,Minnesota Timberwolves,2,1,28,...,2,-20,15,1,0,20180415,HOU,MIN,2017-18,0041700141


In [15]:
shots_df, finals_summary = finals_shots_last_10_seasons("Bam Adebayo")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2017-18                   5
1  2019-20                  19
2  2020-21                   4
3  2021-22                  18
4  2022-23                  23
5  2023-24                   5
6  2024-25                   4
Total Finals shot attempts rows: 975


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041700121,559,1628389,Bam Adebayo,1610612748,Miami Heat,3,1,3,...,2,17,13,1,0,20180414,PHI,MIA,2017-18,0041700121
1,Shot Chart Detail,0041700121,690,1628389,Bam Adebayo,1610612748,Miami Heat,4,4,46,...,1,16,5,1,1,20180414,PHI,MIA,2017-18,0041700121
2,Shot Chart Detail,0041700121,695,1628389,Bam Adebayo,1610612748,Miami Heat,4,4,17,...,2,23,11,1,1,20180414,PHI,MIA,2017-18,0041700121
3,Shot Chart Detail,0041700122,181,1628389,Bam Adebayo,1610612748,Miami Heat,2,11,21,...,0,0,-6,1,0,20180416,PHI,MIA,2017-18,0041700122
4,Shot Chart Detail,0041700122,183,1628389,Bam Adebayo,1610612748,Miami Heat,2,11,20,...,0,0,-6,1,0,20180416,PHI,MIA,2017-18,0041700122
5,Shot Chart Detail,0041700122,185,1628389,Bam Adebayo,1610612748,Miami Heat,2,11,19,...,0,0,-6,1,0,20180416,PHI,MIA,2017-18,0041700122
6,Shot Chart Detail,0041700122,382,1628389,Bam Adebayo,1610612748,Miami Heat,2,0,1,...,46,129,448,1,0,20180416,PHI,MIA,2017-18,0041700122
7,Shot Chart Detail,0041700123,250,1628389,Bam Adebayo,1610612748,Miami Heat,2,7,51,...,1,5,15,1,1,20180419,MIA,PHI,2017-18,0041700123
8,Shot Chart Detail,0041700123,360,1628389,Bam Adebayo,1610612748,Miami Heat,2,1,58,...,1,13,15,1,0,20180419,MIA,PHI,2017-18,0041700123
9,Shot Chart Detail,0041700123,384,1628389,Bam Adebayo,1610612748,Miami Heat,2,0,2,...,9,96,27,1,1,20180419,MIA,PHI,2017-18,0041700123


In [7]:
name = "Stephen Curry"

player = players.find_players_by_full_name(name)[0]
player_id = player['id']

df = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]

print(df[['DISPLAY_FIRST_LAST', 'HEIGHT', 'WEIGHT']].iloc[0])

DISPLAY_FIRST_LAST    Stephen Curry
HEIGHT                          6-2
WEIGHT                          185
Name: 0, dtype: str
